In [61]:
#pip install TensorFlow

In [28]:
from keras.models import Sequential, load_model
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [29]:
shakespeare_url='https://homl.info/shakespeare'
filepath=tf.keras.utils.get_file('shakespeare_txt',shakespeare_url)
with open(filepath) as f:
    shakespeare_txt=f.read()

In [30]:
shakespeare_txt[0:15]

'First Citizen:\n'

#Convert the text into Tokens to derive a number from each char

In [31]:
tokenizer_char = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer_char.fit_on_texts([shakespeare_txt]) 

print(tokenizer_char.texts_to_sequences(['First']))   #On one example we use texts_to_sequences function 

print(tokenizer_char.sequences_to_texts([[20, 6, 9, 8, 3]]))  #On one example we use sequences_to_texts function

#Number of distinct characters
max_id = len(tokenizer_char.word_index)
print(max_id)

#number of characters
dataset_size=tokenizer_char.document_count
print(dataset_size)

#Encoding each characters by its ID (0 to 38)
[encoded] = np.array(tokenizer_char.texts_to_sequences([shakespeare_txt]))
encoded[0:10]

[[20, 6, 9, 8, 3]]
['f i r s t']
39
1


array([20,  6,  9,  8,  3,  1, 19,  6,  3,  6])

In [10]:
tokenizer_word = keras.preprocessing.text.Tokenizer()
tokenizer_word.fit_on_texts([shakespeare_txt]) 

print(tokenizer_word.texts_to_sequences(['First']))

print(tokenizer_word.sequences_to_texts([[88,269]]))

#Number of distinct characters
max_id = len(tokenizer_word.word_index)
print(max_id)

#number of characters
dataset_size=tokenizer_word.document_count
print(dataset_size)

#Encoding each word by its ID
[encoded] = np.array(tokenizer_word.texts_to_sequences([shakespeare_txt]))
[encoded]

[[88]]
['first citizen']
12632
1


[array([  88,  269,  139, ...,   25,  131, 1828])]

In [36]:
dataset_size = encoded.shape[0]
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

#Determine the window size for each segment - First window ranges from 0 to 100, second window ranges from 1 to 101 and so on
n_steps = 100
window_length = n_steps + 1               #Target = Input shifted 1 character
dataset = dataset.window(window_length, shift=1, drop_remainder= True)

#Window method creates a nested dataset and so we need to use flatten map to transform it
dataset=dataset.flat_map(lambda window: window.batch(window_length))

#Gradient Descent works best when data is independent and identically distributed, so data is shuffled
batch_size = 32
dataset=dataset.shuffle(10000).batch(batch_size)

#Divide the dataset into train and target batch window data
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

#Convert the training dataset into one-hot-encoding vector
dataset=dataset.map(lambda X_batch,y_batch: (tf.one_hot(X_batch, depth = max_id), y_batch))
dataset = dataset.prefetch(1)
dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 39), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int32, name=None))>

After performing all the preprocessing tasks, now we can train and compile the RNN model and start predicting the output

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation='softmax'))
])
model.summary()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam')
history = model.fit(dataset, epochs=2)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_26 (GRU)                (None, None, 128)         64896     
                                                                 
 gru_27 (GRU)                (None, None, 128)         99072     
                                                                 
 time_distributed_13 (TimeDi  (None, None, 39)         5031      
 stributed)                                                      
                                                                 
Total params: 168,999
Trainable params: 168,999
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
   6630/Unknown - 509s 76ms/step - loss: 1.7466